* The following code is modified based on the original torchcv project. 
* We are going to use PASACAL VOC12 as dataset.
* You could donwload VOC2012 
  train/validation: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/#devkit
* test data: https://pjreddie.com/projects/pascal-voc-dataset-mirror/


In [1]:
import os
import random
import argparse
import matplotlib.pyplot as plt
from PIL import Image
from time import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as T

from __future__ import print_function
from dbpn import Net as DBPN
from dbpn import get_pair_set
from ssd import SSD
from ssd import build_ssd
from ssd.layers.modules import MultiBoxLoss
from ssd.data.config import voc
from ssd.data import detection_collate
from ssd.data import VOCAnnotationTransform, VOCDetection, BaseTransform, SRDetection

import sys; sys.argv=['']; del sys


In [2]:
# Arguments & settings
parser = argparse.ArgumentParser(description='PyTorch Super Resolution Detection Networks')
parser.add_argument('--upscale_factor', type=int, default=4, help="super resolution upscale factor")
parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
parser.add_argument('--batch_size', type=int, default=4, help='training batch size') # GPU: 9GB!
parser.add_argument('--threads', type=int, default=4, help='number of threads for data loading')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')
parser.add_argument('--gpu_mode', type=bool, default=True)
parser.add_argument('--gpus', default=1, type=float, help='number of gpu')
#parser.add_argument('--test_dataset', type=str, default='VOC12-LR-X8-test')
#parser.add_argument('--sr_dataset', type=str, default='VOC12-SR-X8')
parser.add_argument('--train_dataset', type=str, default='VOC12-LR-x4')
parser.add_argument('--hr_dataset', type=str, default='VOC12-HR')
parser.add_argument('--anno_path', type=str, default='Annotations')
parser.add_argument('--imSetpath', type=str, default='ImageSets')
parser.add_argument('--input_dir', type=str, default='./dataset')
#parser.add_argument('--output', default='./dataset/results', help='Location to save some outputs')
parser.add_argument('--checkpt', default='./checkpoint', help='Location to save checkpoint models')
parser.add_argument('--lr', type=float, default=1e-4, help='Learning Rate. Default=0.0001')
parser.add_argument('--nEpochs', type=int, default=5, help='number of epochs to fine tune net S over target loss')

opt = parser.parse_args()

gpus_list=range(opt.gpus)
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)
if cuda:
    torch.cuda.manual_seed(opt.seed)
    torch.cuda.empty_cache()


Namespace(anno_path='Annotations', batch_size=4, checkpt='./checkpoint', gpu_mode=True, gpus=1, hr_dataset='VOC12-HR', imSetpath='ImageSets', input_dir='./dataset', lr=0.0001, nEpochs=5, seed=123, testBatchSize=1, threads=4, train_dataset='VOC12-LR-x4', upscale_factor=4)


In [3]:
print('===> Loading net S fine-tune training datasets')
# VOC0712 dataset mean
MEANS = (104, 117, 123)

sd_dataset = SRDetection(root='./dataset', image_sets='trainval.txt', data_mean = MEANS,
                         target_transform = VOCAnnotationTransform())

sd_data_loader = DataLoader(dataset=sd_dataset, batch_size=opt.batch_size, num_workers=opt.threads,
                            shuffle=False, collate_fn=detection_collate, pin_memory=True)


===> Loading net S fine-tune training datasets


In [4]:
# TODO:
# 1. come up with test/eval functions
# 2. come up with visualization image

class DBPN2SSD(nn.Module):
    
    def __init__(self, s_model_name, d_model_name, d_frozen):
        super(DBPN2SSD, self).__init__()
        self.supervis = DBPN(num_channels=3, base_filter=64, feat=256, num_stages=7, scale_factor=4)
        if os.path.exists(s_model_name):
            self.supervis = torch.nn.DataParallel(self.supervis, device_ids=gpus_list)
            self.supervis.load_state_dict(torch.load(s_model_name, map_location=lambda storage, loc: storage))

        # self.detector = SSD(), setup ssd as 'train' mode for gradient flow
        # later at test/eval situation, we will overwrite it's mode to 'test'
        self.detector = build_ssd('train', 300, 21)
        if os.path.exists(d_model_name):
            self.detector.load_state_dict(torch.load(d_model_name, map_location=lambda storage, loc: storage))
        if d_frozen:
            for param in self.detector.parameters():
                param.requires_grad = False
        
    def forward(self, x):
        superx = self.supervis(x)
        # current detector: SSD300, so assume superx: 300x300!
        detect = self.detector(superx)
        return (superx, detect)



In [5]:
# Hyperparameters - balancing weights of net S loss vs. net D loss
alpha = 1.0
beta = 1.0

decay_steps = [5000, 10000, 20000, 40000, 80000, 160000]
gamma = 0.1
def adjust_learning_rate(optimizer, gamma=0.1, step=0):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = opt.lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def train():
    #SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    #SDnet = DBPN2SSD('checkpoint/SRx4_4000.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    SDnet = DBPN2SSD('dbpn/models/VOC12-LR-x4-DBPN-ep100.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    
    net = SDnet

    # Criterions:
    #
    # net SR loss function, change to L2 later
    criterion_sr = nn.MSELoss()

    # ssd loss function
    criterion_ssd = MultiBoxLoss(voc['num_classes'], 0.5, True, 0, True, 3, 0.5, False, use_gpu=cuda)

    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        #net.supervis = torch.nn.DataParallel(net.supervis)
        cudnn.benchmark = True
        net = net.cuda(gpus_list[0])
        criterion_sr = criterion_sr.cuda(gpus_list[0])
        criterion_ssd = criterion_ssd.cuda(gpus_list[0])


    # optimizer = optim.SGD(net.supervis.parameters(), lr=opt.lr, \
    #                       momentum=opt.momentum, weight_decay=opt.weight_decay)
    #
    # Use universal Adam first :)
    # we can be Specific to what part of network's parameters to optimize, here is SuperR Net
    optimizer = optim.Adam(net.supervis.parameters(), lr=opt.lr, betas=(0.9, 0.999), eps=1e-8)
    epoch_loss = 0
    
    # set model to training mode
    net.train()
    
    epoch = 1
    max_iter = 20000
    decay_step = 0
    for iteration, batch in enumerate(sd_data_loader, 1):
        
        if iteration == max_iter:
            break
        if iteration in decay_steps:
            decay_step += 1
            adjust_learning_rate(optimizer, gamma, decay_step)
            
        # input is LR image; sr_target is pseudo (300x300) original HR image, det_target is bboxes
        input, sr_target, det_target = batch[0], batch[1], batch[2]
        if cuda:
            input = Variable(input.cuda(gpus_list[0]))
            sr_target = Variable(sr_target.cuda(gpus_list[0]))
            det_target = [Variable(ann.cuda(gpus_list[0]), volatile=True) for ann in det_target]
        else:
            input = Variable(input)
            sr_target = Variable(sr_target)
            det_target = [Variable(ann, volatile=True) for ann in det_target]

        optimizer.zero_grad()
        t0 = time()
        
        sr_out,ssd_out = net(input)
        loss_sr = criterion_sr(sr_out, sr_target)
        
        # Compound Loss from net-S: loss_sr and net-D: loss_l, loss_c
        loss_l, loss_c = criterion_ssd(ssd_out, det_target)
        loss = loss_sr * alpha + beta *(loss_l + loss_c)
        
        epoch_loss += loss.data[0]
        loss.backward()
        optimizer.step()
        t1 = time()

        if iteration != 0 and iteration % 2000 == 0:
            print('Saving state, iter:', iteration)
            torch.save(net.supervis.state_dict(), opt.checkpt + '/' + 'SRx4_' + repr(iteration+4000) + '.pth')
        
        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, \
                    iteration, len(sd_data_loader), loss.data[0], (t1 - t0)))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(sd_data_loader)))


In [6]:
train()


/home/x/cs231n/super-resolution-detection/dbpn/dbpn.py:47: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/home/x/cs231n/super-resolution-detection/dbpn/dbpn.py:51: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/home/x/cs231n/super-resolution-detection/ssd/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/x/cs231n/super-resolution-detection/ssd/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)
__main__:69: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:85: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use 

===> Epoch[1](1/2885): Loss: 79177120.0000 || Timer: 4.1878 sec.
===> Epoch[1](2/2885): Loss: 6594037.5000 || Timer: 0.6564 sec.
===> Epoch[1](3/2885): Loss: 8515957.0000 || Timer: 0.6989 sec.
===> Epoch[1](4/2885): Loss: 3454282.7500 || Timer: 0.6480 sec.
===> Epoch[1](5/2885): Loss: 1422411.0000 || Timer: 0.6553 sec.
===> Epoch[1](6/2885): Loss: 2457505.5000 || Timer: 0.6412 sec.
===> Epoch[1](7/2885): Loss: 1356234.7500 || Timer: 0.6470 sec.
===> Epoch[1](8/2885): Loss: 1752461.3750 || Timer: 0.6402 sec.
===> Epoch[1](9/2885): Loss: 2055880.2500 || Timer: 0.6497 sec.
===> Epoch[1](10/2885): Loss: 688784.6250 || Timer: 0.6418 sec.
===> Epoch[1](11/2885): Loss: 590908.5625 || Timer: 0.6512 sec.
===> Epoch[1](12/2885): Loss: 292935.3125 || Timer: 0.6434 sec.
===> Epoch[1](13/2885): Loss: 252475.6406 || Timer: 0.6408 sec.
===> Epoch[1](14/2885): Loss: 144002.0156 || Timer: 0.6422 sec.
===> Epoch[1](15/2885): Loss: 227901.3906 || Timer: 0.6528 sec.
===> Epoch[1](16/2885): Loss: 117679.39

===> Epoch[1](131/2885): Loss: 15062.8896 || Timer: 0.6518 sec.
===> Epoch[1](132/2885): Loss: 13764.1836 || Timer: 0.6510 sec.
===> Epoch[1](133/2885): Loss: 12224.9561 || Timer: 0.6591 sec.
===> Epoch[1](134/2885): Loss: 21935.2891 || Timer: 0.6520 sec.
===> Epoch[1](135/2885): Loss: 15459.1562 || Timer: 0.6502 sec.
===> Epoch[1](136/2885): Loss: 14007.4170 || Timer: 0.6642 sec.
===> Epoch[1](137/2885): Loss: 19351.6172 || Timer: 0.6522 sec.
===> Epoch[1](138/2885): Loss: 5409.5010 || Timer: 0.6595 sec.
===> Epoch[1](139/2885): Loss: 14291.8096 || Timer: 0.6602 sec.
===> Epoch[1](140/2885): Loss: 20817.6094 || Timer: 0.6529 sec.
===> Epoch[1](141/2885): Loss: 10484.3691 || Timer: 0.6548 sec.
===> Epoch[1](142/2885): Loss: 12753.0547 || Timer: 0.6532 sec.
===> Epoch[1](143/2885): Loss: 15456.9307 || Timer: 0.6507 sec.
===> Epoch[1](144/2885): Loss: 4179.6196 || Timer: 0.6526 sec.
===> Epoch[1](145/2885): Loss: 9690.4785 || Timer: 0.6587 sec.
===> Epoch[1](146/2885): Loss: 11320.0576 |

===> Epoch[1](260/2885): Loss: 7801.3936 || Timer: 0.6776 sec.
===> Epoch[1](261/2885): Loss: 9758.3584 || Timer: 0.6706 sec.
===> Epoch[1](262/2885): Loss: 8864.1934 || Timer: 0.6866 sec.
===> Epoch[1](263/2885): Loss: 11882.8945 || Timer: 0.6528 sec.
===> Epoch[1](264/2885): Loss: 5846.4316 || Timer: 0.6973 sec.
===> Epoch[1](265/2885): Loss: 7353.9087 || Timer: 0.6891 sec.
===> Epoch[1](266/2885): Loss: 7128.0303 || Timer: 0.6787 sec.
===> Epoch[1](267/2885): Loss: 6504.9570 || Timer: 0.6912 sec.
===> Epoch[1](268/2885): Loss: 5950.2837 || Timer: 0.6998 sec.
===> Epoch[1](269/2885): Loss: 6060.7383 || Timer: 0.7081 sec.
===> Epoch[1](270/2885): Loss: 7488.2749 || Timer: 0.6976 sec.
===> Epoch[1](271/2885): Loss: 5093.9648 || Timer: 0.6689 sec.
===> Epoch[1](272/2885): Loss: 7114.5830 || Timer: 0.6708 sec.
===> Epoch[1](273/2885): Loss: 9032.7686 || Timer: 0.6605 sec.
===> Epoch[1](274/2885): Loss: 3784.1362 || Timer: 0.6661 sec.
===> Epoch[1](275/2885): Loss: 9194.7949 || Timer: 0.6

===> Epoch[1](391/2885): Loss: 5440.6670 || Timer: 0.6695 sec.
===> Epoch[1](392/2885): Loss: 3712.0315 || Timer: 0.7021 sec.
===> Epoch[1](393/2885): Loss: 6568.3125 || Timer: 0.6913 sec.
===> Epoch[1](394/2885): Loss: 4947.2812 || Timer: 0.7021 sec.
===> Epoch[1](395/2885): Loss: 5080.8242 || Timer: 0.6664 sec.
===> Epoch[1](396/2885): Loss: 4495.0684 || Timer: 0.6945 sec.
===> Epoch[1](397/2885): Loss: 5981.8687 || Timer: 0.6652 sec.
===> Epoch[1](398/2885): Loss: 5615.8872 || Timer: 0.6633 sec.
===> Epoch[1](399/2885): Loss: 4162.8638 || Timer: 0.7063 sec.
===> Epoch[1](400/2885): Loss: 9654.7119 || Timer: 0.6707 sec.
===> Epoch[1](401/2885): Loss: 5108.2759 || Timer: 0.6812 sec.
===> Epoch[1](402/2885): Loss: 3514.2944 || Timer: 0.7025 sec.
===> Epoch[1](403/2885): Loss: 4995.0425 || Timer: 0.6831 sec.
===> Epoch[1](404/2885): Loss: 4644.8916 || Timer: 0.7165 sec.
===> Epoch[1](405/2885): Loss: 3902.4221 || Timer: 0.6883 sec.
===> Epoch[1](406/2885): Loss: 4981.2280 || Timer: 0.65

...

===> Epoch[1](2337/2885): Loss: 973.2831 || Timer: 0.7021 sec.
===> Epoch[1](2338/2885): Loss: 679.5805 || Timer: 0.7148 sec.
===> Epoch[1](2339/2885): Loss: 1086.9784 || Timer: 0.7030 sec.
===> Epoch[1](2340/2885): Loss: 828.8893 || Timer: 0.6861 sec.
===> Epoch[1](2341/2885): Loss: 752.3912 || Timer: 0.6738 sec.
===> Epoch[1](2342/2885): Loss: 917.2324 || Timer: 0.6639 sec.
===> Epoch[1](2343/2885): Loss: 878.4651 || Timer: 0.6663 sec.
===> Epoch[1](2344/2885): Loss: 1132.3788 || Timer: 0.6701 sec.
===> Epoch[1](2345/2885): Loss: 1009.5322 || Timer: 0.6653 sec.
===> Epoch[1](2346/2885): Loss: 776.2277 || Timer: 0.6623 sec.
===> Epoch[1](2347/2885): Loss: 1005.9977 || Timer: 0.6693 sec.
===> Epoch[1](2348/2885): Loss: 1005.4313 || Timer: 0.7968 sec.
===> Epoch[1](2349/2885): Loss: 798.4933 || Timer: 0.6907 sec.
===> Epoch[1](2350/2885): Loss: 803.2390 || Timer: 0.7025 sec.
===> Epoch[1](2351/2885): Loss: 878.9828 || Timer: 0.6895 sec.
===> Epoch[1](2352/2885): Loss: 501.4494 || Timer:

===> Epoch[1](2467/2885): Loss: 947.9462 || Timer: 0.6672 sec.
===> Epoch[1](2468/2885): Loss: 580.5750 || Timer: 0.6580 sec.
===> Epoch[1](2469/2885): Loss: 1008.1902 || Timer: 0.6596 sec.
===> Epoch[1](2470/2885): Loss: 816.2084 || Timer: 0.6606 sec.
===> Epoch[1](2471/2885): Loss: 989.6187 || Timer: 0.6613 sec.
===> Epoch[1](2472/2885): Loss: 896.2933 || Timer: 0.6555 sec.
===> Epoch[1](2473/2885): Loss: 1233.6376 || Timer: 0.6585 sec.
===> Epoch[1](2474/2885): Loss: 651.1503 || Timer: 0.6671 sec.
===> Epoch[1](2475/2885): Loss: 784.7142 || Timer: 0.6728 sec.
===> Epoch[1](2476/2885): Loss: 509.4995 || Timer: 0.6791 sec.
===> Epoch[1](2477/2885): Loss: 653.2936 || Timer: 0.6805 sec.
===> Epoch[1](2478/2885): Loss: 499.9443 || Timer: 0.6770 sec.
===> Epoch[1](2479/2885): Loss: 949.2417 || Timer: 0.6588 sec.
===> Epoch[1](2480/2885): Loss: 1004.2197 || Timer: 0.6666 sec.
===> Epoch[1](2481/2885): Loss: 784.7683 || Timer: 0.6957 sec.
===> Epoch[1](2482/2885): Loss: 690.6932 || Timer: 0

===> Epoch[1](2597/2885): Loss: 848.8642 || Timer: 0.6565 sec.
===> Epoch[1](2598/2885): Loss: 1003.8631 || Timer: 0.6560 sec.
===> Epoch[1](2599/2885): Loss: 1097.4636 || Timer: 0.6625 sec.
===> Epoch[1](2600/2885): Loss: 756.4975 || Timer: 0.6627 sec.
===> Epoch[1](2601/2885): Loss: 1212.9028 || Timer: 0.6606 sec.
===> Epoch[1](2602/2885): Loss: 975.1480 || Timer: 0.6759 sec.
===> Epoch[1](2603/2885): Loss: 1299.2130 || Timer: 0.6594 sec.
===> Epoch[1](2604/2885): Loss: 997.4528 || Timer: 0.6539 sec.
===> Epoch[1](2605/2885): Loss: 978.2876 || Timer: 0.6592 sec.
===> Epoch[1](2606/2885): Loss: 624.5370 || Timer: 0.6689 sec.
===> Epoch[1](2607/2885): Loss: 816.1769 || Timer: 0.6547 sec.
===> Epoch[1](2608/2885): Loss: 956.1588 || Timer: 0.6564 sec.
===> Epoch[1](2609/2885): Loss: 1253.5496 || Timer: 0.6732 sec.
===> Epoch[1](2610/2885): Loss: 724.6188 || Timer: 0.6788 sec.
===> Epoch[1](2611/2885): Loss: 869.7845 || Timer: 0.6650 sec.
===> Epoch[1](2612/2885): Loss: 685.5538 || Timer:

===> Epoch[1](2727/2885): Loss: 971.6696 || Timer: 0.6599 sec.
===> Epoch[1](2728/2885): Loss: 682.0369 || Timer: 0.6648 sec.
===> Epoch[1](2729/2885): Loss: 708.3339 || Timer: 0.6939 sec.
===> Epoch[1](2730/2885): Loss: 796.5333 || Timer: 0.6760 sec.
===> Epoch[1](2731/2885): Loss: 523.6095 || Timer: 0.6706 sec.
===> Epoch[1](2732/2885): Loss: 798.0642 || Timer: 0.6642 sec.
===> Epoch[1](2733/2885): Loss: 438.7881 || Timer: 0.6616 sec.
===> Epoch[1](2734/2885): Loss: 943.0986 || Timer: 0.6698 sec.
===> Epoch[1](2735/2885): Loss: 535.3674 || Timer: 0.6549 sec.
===> Epoch[1](2736/2885): Loss: 859.9691 || Timer: 0.6634 sec.
===> Epoch[1](2737/2885): Loss: 632.9286 || Timer: 0.6622 sec.
===> Epoch[1](2738/2885): Loss: 926.9600 || Timer: 0.6950 sec.
===> Epoch[1](2739/2885): Loss: 692.7825 || Timer: 0.6943 sec.
===> Epoch[1](2740/2885): Loss: 772.3000 || Timer: 0.7052 sec.
===> Epoch[1](2741/2885): Loss: 948.5859 || Timer: 0.6777 sec.
===> Epoch[1](2742/2885): Loss: 1010.8386 || Timer: 0.6

===> Epoch[1](2857/2885): Loss: 708.7996 || Timer: 0.6577 sec.
===> Epoch[1](2858/2885): Loss: 646.6881 || Timer: 0.6624 sec.
===> Epoch[1](2859/2885): Loss: 388.2649 || Timer: 0.6794 sec.
===> Epoch[1](2860/2885): Loss: 712.5794 || Timer: 0.6824 sec.
===> Epoch[1](2861/2885): Loss: 700.2434 || Timer: 0.6650 sec.
===> Epoch[1](2862/2885): Loss: 900.2536 || Timer: 0.6770 sec.
===> Epoch[1](2863/2885): Loss: 943.8935 || Timer: 0.6627 sec.
===> Epoch[1](2864/2885): Loss: 685.2469 || Timer: 0.6628 sec.
===> Epoch[1](2865/2885): Loss: 1137.8455 || Timer: 0.6619 sec.
===> Epoch[1](2866/2885): Loss: 678.5848 || Timer: 0.6609 sec.
===> Epoch[1](2867/2885): Loss: 967.7050 || Timer: 0.6560 sec.
===> Epoch[1](2868/2885): Loss: 668.1830 || Timer: 0.6555 sec.
===> Epoch[1](2869/2885): Loss: 953.6281 || Timer: 0.6561 sec.
===> Epoch[1](2870/2885): Loss: 723.5200 || Timer: 0.6635 sec.
===> Epoch[1](2871/2885): Loss: 605.9244 || Timer: 0.6533 sec.
===> Epoch[1](2872/2885): Loss: 563.4431 || Timer: 0.6

In [ ]:
torch.cuda.empty_cache()


### Debugging code below, no need to run :)

In [ ]:
print(' YOU Donot need to run this ===> Loading some datasets')

lr_path = os.path.join(opt.input_dir, opt.train_dataset)
hr_path = os.path.join(opt.input_dir, opt.hr_dataset)

fine_train_set = get_pair_set(lr_path, hr_path)
train_data_loader = DataLoader(dataset=fine_train_set, num_workers=opt.threads, \
                               batch_size=opt.testBatchSize, shuffle=False)


In [ ]:
def simple_test():
    SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    net = SDnet
    
    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
        net = net.cuda()

    else:
        torch.set_default_tensor_type('torch.FloatTensor')


    
simple_test()

In [ ]:
# dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor ## UNCOMMENT THIS LINE IF YOU'RE ON A GPU!

# Real samples were all associated with Ture labels - 1
T_labels = torch.ones(logits_real.size()).type(dtype)
# Fake samples were all associated with Fake labels - 0
F_labels = torch.zeros(logits_fake.size()).type(dtype)
